In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO
import cv2
import os

# Load model
model = YOLO('drive/MyDrive/runs-trees/detect/finetune_with_trees/weights/best.pt')

# Open the input video
video_path = "Vid.mp4"
cap = cv2.VideoCapture(video_path)

# Create directory to save images
output_dir = "predicted_frames"
os.makedirs(output_dir, exist_ok=True)

frame_count = 0
saved_count = 0

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if frame_count % 20 == 0:
        # Run prediction
        results = model.predict(source=frame, conf=0.5, verbose=False)
        result = results[0]

        # Draw bounding boxes without scores
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            label = model.names[cls_id]

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Save the frame as an image
        output_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(output_path, frame)
        saved_count += 1

# Release everything
cap.release()

print(f"Saved {saved_count} predicted frames in '{output_dir}/'")


Saved 42 predicted frames in 'predicted_frames/'


In [ ]:
from ultralytics import YOLO
import cv2
import os
import csv

# Load model
model = YOLO('drive/MyDrive/runs-trees/detect/finetune_with_trees/weights/best.pt')

# Open the input video
video_path = "Vid.mp4"
cap = cv2.VideoCapture(video_path)

# Output directories and CSV path
output_dir = "predicted_frames"
os.makedirs(output_dir, exist_ok=True)

csv_path = os.path.join(output_dir, "bounding_boxes.csv")
csv_file = open(csv_path, mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["frame_number", "class_name", "x1", "y1", "x2", "y2", "confidence"])

frame_count = 0
saved_count = 0

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    if frame_count % 20 == 0:
        # Run prediction
        results = model.predict(source=frame, conf=0.5, verbose=False)
        result = results[0]

        # Draw and save predictions
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            confidence = float(box.conf[0])
            label = model.names[cls_id]

            # Draw on frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Write to CSV
            csv_writer.writerow([frame_count, label, x1, y1, x2, y2, round(confidence, 4)])

        # Save frame as image
        output_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(output_path, frame)
        saved_count += 1

# Cleanup
cap.release()
csv_file.close()

print(f"Saved {saved_count} predicted frames and CSV file at '{csv_path}'")


Saved 42 predicted frames and CSV file at 'predicted_frames/bounding_boxes.csv'


In [ ]:
def save_every_20th_frame(video_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_idx = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % 20 == 0:
            frame_filename = f"frame_{frame_idx}.jpg"
            save_path = os.path.join(output_dir, frame_filename)
            cv2.imwrite(save_path, frame)
            print(f"✅ Saved {frame_filename}")
            saved_count += 1

        frame_idx += 1

    cap.release()
    print(f"\n🎉 Done. Saved {saved_count} frames to '{output_dir}'")
save_every_20th_frame("Vid.mp4", "blank_frames")

✅ Saved frame_0.jpg
✅ Saved frame_20.jpg
✅ Saved frame_40.jpg
✅ Saved frame_60.jpg
✅ Saved frame_80.jpg
✅ Saved frame_100.jpg
✅ Saved frame_120.jpg
✅ Saved frame_140.jpg
✅ Saved frame_160.jpg
✅ Saved frame_180.jpg
✅ Saved frame_200.jpg
✅ Saved frame_220.jpg
✅ Saved frame_240.jpg
✅ Saved frame_260.jpg
✅ Saved frame_280.jpg
✅ Saved frame_300.jpg
✅ Saved frame_320.jpg
✅ Saved frame_340.jpg
✅ Saved frame_360.jpg
✅ Saved frame_380.jpg
✅ Saved frame_400.jpg
✅ Saved frame_420.jpg
✅ Saved frame_440.jpg
✅ Saved frame_460.jpg
✅ Saved frame_480.jpg
✅ Saved frame_500.jpg
✅ Saved frame_520.jpg
✅ Saved frame_540.jpg
✅ Saved frame_560.jpg
✅ Saved frame_580.jpg
✅ Saved frame_600.jpg
✅ Saved frame_620.jpg
✅ Saved frame_640.jpg
✅ Saved frame_660.jpg
✅ Saved frame_680.jpg
✅ Saved frame_700.jpg
✅ Saved frame_720.jpg
✅ Saved frame_740.jpg
✅ Saved frame_760.jpg
✅ Saved frame_780.jpg
✅ Saved frame_800.jpg
✅ Saved frame_820.jpg
✅ Saved frame_840.jpg

🎉 Done. Saved 43 frames to 'blank_frames'


In [ ]:
import pandas as pd
import json
import csv

def convert_via_to_xyxy(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    output_rows = []

    for _, row in df.iterrows():
        filename = row['filename']
        shape_str = row['region_shape_attributes']

        try:
            shape = json.loads(shape_str.replace("'", '"'))
        except Exception as e:
            print(f"Skipping row: {e}")
            continue

        if shape.get('name') != 'rect':
            continue

        try:
            x1 = int(shape['x'])
            y1 = int(shape['y'])
            x2 = x1 + int(shape['width'])
            y2 = y1 + int(shape['height'])
            output_rows.append([filename, x1, y1, x2, y2])
        except Exception as e:
            print(f"Bad shape row: {e}")
            continue

    with open(output_csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['filename', 'x1', 'y1', 'x2', 'y2'])
        writer.writerows(output_rows)

    print(f"Saved static parking lot to: {output_csv_path}")

convert_via_to_xyxy('Video_Script2_csv.csv', 'parking_spots2.csv')


Saved static parking lot to: parking_spots2.csv


In [ ]:
import pandas as pd
import json

 # From VGG annotator
vgg_df = pd.read_csv("parking_spots2.csv")
detections_df = pd.read_csv("predicted_frames/bounding_boxes.csv")  # From YOLO model

# IOU function
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = inter_area / float(boxA_area + boxB_area - inter_area)
    return iou

# Set IOU threshold for "occupied"
IOU_THRESHOLD = 0.1

# Prepare output
results = []


# Iterate per frame
# Group detections by frame
grouped_detections = detections_df.groupby("frame_number")

results = []

for frame_num, detections in grouped_detections:
    # Get only the parking spots corresponding to this frame's filename
    frame_spots = vgg_df

    for spot_id, spot in frame_spots.reset_index(drop=True).iterrows():
        spot_box = [spot.x1, spot.y1, spot.x2, spot.y2]
        occupied = False

        for _, car in detections.iterrows():
            car_box = [car.x1, car.y1, car.x2, car.y2]
            iou = compute_iou(spot_box, car_box)
            if iou >= IOU_THRESHOLD:
                occupied = True
                break

        results.append({
            "frame_number": frame_num,
            "spot_id": spot_id,
            "status": "occupied" if occupied else "vacant",
            "Coordinates" : car_box
        })


# Save results
results_df = pd.DataFrame(results)
results_df.to_csv("parking_occupancy_status.csv", index=False)

print("Done. Results saved to 'parking_occupancy_status.csv'")


Done. Results saved to 'parking_occupancy_status.csv'


In [ ]:
import pandas as pd
import cv2
import os

# Load data
vgg_df = pd.read_csv("parking_spots2.csv")
detections_df = pd.read_csv("predicted_frames/bounding_boxes.csv")

# IOU function
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxA_area = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxB_area = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = inter_area / float(boxA_area + boxB_area - inter_area)
    return iou

IOU_THRESHOLD = 0.1
grouped_detections = detections_df.groupby("frame_number")
results = []

# Make sure output folder exists
frame_folder = "blank_frames"  # where frame_20.jpg, etc. are
output_folder = "output_visuals3"
os.makedirs(output_folder, exist_ok=True)

for frame_num, detections in grouped_detections:
    frame_spots = vgg_df
    image_name = f"frame_{frame_num}.jpg"
    image_path = os.path.join(frame_folder, image_name)
    img = cv2.imread(image_path)

    if img is None:
        print(f"⚠️ Skipping {image_name} (not found or unreadable)")
        continue

    for spot_id, spot in frame_spots.reset_index(drop=True).iterrows():
        spot_box = [spot.x1, spot.y1, spot.x2, spot.y2]
        occupied = False

        for _, car in detections.iterrows():
            car_box = [car.x1, car.y1, car.x2, car.y2]
            iou = compute_iou(spot_box, car_box)
            if iou >= IOU_THRESHOLD:
                occupied = True
                break

        color = (0, 0, 255) if occupied else (0, 255, 0)  # red or green
        label = f""
        cv2.rectangle(img, (spot.x1, spot.y1), (spot.x2, spot.y2), color, 2)
        cv2.putText(img, label, (spot.x1, spot.y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        results.append({
            "frame_number": frame_num,
            "spot_id": spot_id,
            "status": "occupied" if occupied else "vacant",
             "Coordinates" : car_box
        })

    # Save the output image
    out_path = os.path.join(output_folder, image_name)
    cv2.imwrite(out_path, img)
    print(f"✅ Saved visualization: {out_path}")

# Save CSV
results_df = pd.DataFrame(results)
results_df.to_csv("parking_occupancy_status3.csv", index=False)
print("\n🎉 All frames processed. Results saved.")


✅ Saved visualization: output_visuals3/frame_20.jpg
✅ Saved visualization: output_visuals3/frame_40.jpg
✅ Saved visualization: output_visuals3/frame_60.jpg
✅ Saved visualization: output_visuals3/frame_80.jpg
✅ Saved visualization: output_visuals3/frame_100.jpg
✅ Saved visualization: output_visuals3/frame_120.jpg
✅ Saved visualization: output_visuals3/frame_140.jpg
✅ Saved visualization: output_visuals3/frame_160.jpg
✅ Saved visualization: output_visuals3/frame_180.jpg
✅ Saved visualization: output_visuals3/frame_200.jpg
✅ Saved visualization: output_visuals3/frame_220.jpg
✅ Saved visualization: output_visuals3/frame_240.jpg
✅ Saved visualization: output_visuals3/frame_260.jpg
✅ Saved visualization: output_visuals3/frame_280.jpg
✅ Saved visualization: output_visuals3/frame_300.jpg
✅ Saved visualization: output_visuals3/frame_320.jpg
✅ Saved visualization: output_visuals3/frame_340.jpg
✅ Saved visualization: output_visuals3/frame_360.jpg
✅ Saved visualization: output_visuals3/frame_380.j

In [ ]:
import zipfile
def zip_folder(folder_path, output_zip):
    with zipfile.ZipFile(output_zip, 'w') as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                abs_path = os.path.join(root, file)
                rel_path = os.path.relpath(abs_path, folder_path)
                zipf.write(abs_path, arcname=rel_path)

# Example:
zip_folder("output_visuals3", "output_visuals.zip")